In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))
import config
import pandas as pd
import numpy as np
import tqdm

from utils.constants import (
    DATA_TRAIN_DIR, 
    DATA_VALIDATION_DIR, 
    DATA_TEST_NOISE_DIR, 
    DATA_TEST_LANDING_DIR, 
    DATA_TEST_DEPARTING_DIR, 
    DATA_TEST_MANOEUVER_DIR, 
    DIFF_DATA, 
    K_LAG, 
    K_ORDER, 
    DIFF_FEATURES, 
    WINDOW_SIZE_STANDARD_AUTOENCODER, 
    FEATURES
)
from utils.constants import DATA_TRAIN_DIR

from utils.preprocessing import filter_outliers, diff_data, get_windows_data

In [2]:
data_dict = {
    "train": [],
    "validation": [],
    "test_noise": [],
    "test_landing": [],
    "test_departing": [],
    "test_manoeuver": [],
}

In [8]:
paths = {
    "train": DATA_TRAIN_DIR, 
    "validation": DATA_VALIDATION_DIR, 
    "test_noise": DATA_TEST_NOISE_DIR, 
    "test_landing": DATA_TEST_LANDING_DIR, 
    "test_departing": DATA_TEST_DEPARTING_DIR,
    "test_manoeuver": DATA_TEST_MANOEUVER_DIR
}

In [9]:
for key, path in paths.items():
    files = os.listdir(path)
    
    for file in files:
        file_path = os.path.join(path, file)
        
        # Check if the path is a file
        if os.path.isfile(file_path):
            df = pd.read_csv(file_path)
            
            if key == "train":
                df = filter_outliers(df, cols=["longitude", "latitude", "altitude", "groundspeed", "x", "y"], std=5)
            elif key == "validation":
                df = filter_outliers(df, cols=["longitude", "latitude", "altitude", "groundspeed", "x", "y"], std=8)

            if DIFF_DATA:
                df = diff_data(df, cols=DIFF_FEATURES, lag=K_LAG, order=K_ORDER)
            
            data_dict[key].append(df)

In [10]:
keys = [x for x in data_dict.keys() if x.startswith("test")]
keys

['test_noise', 'test_landing', 'test_departing', 'test_manoeuver']

In [15]:
# Initialize lists to store windowed data and labels for test datasets
windowed_test_data_list = []
windowed_test_labels_list = []

# Iterate over each test dataset key
for key in tqdm(keys, desc="Processing test datasets"):
    temp_data_list = []
    temp_labels_list = []
    
    for df in tqdm(data_dict[key], desc=f"Processing {key}"):
        X, y = get_windows_data(df[FEATURES], df["anomaly"], window_size=WINDOW_SIZE_STANDARD_AUTOENCODER, tsfresh=True)
        temp_data_list.append(X)
        temp_labels_list.append(y)
    
    windowed_test_data_list.append(temp_data_list)
    windowed_test_labels_list.append(temp_labels_list)

TypeError: 'module' object is not callable